In [9]:
%run ./work/lib.py

In [10]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [11]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [12]:
TABLE_NAME = 'dim_customer'

In [13]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/customers_dataset.csv'.format(LAKE_HOME), header=True)

df_customer = \
    df.select(
        col('customer_id').cast(StringType()).alias('ID_CUSTOMER'),
        col('customer_zip_code_prefix').cast(StringType()).alias('COD_ZIP'),
        col('customer_city').cast(StringType()).alias('DES_CITY'),
        col('customer_state').cast(StringType()).alias('COD_STATE')
    )

sk = create_sk(spark=spark, df=df_customer, key_column_name='ID_CUSTOMER', table_name=TABLE_NAME)

In [14]:
dim_customer = \
    df_customer \
        .withColumn('SK_CUSTOMER', locate_sk(sk)(col('ID_CUSTOMER')))

In [15]:
dim_customer = \
    dim_customer.select(
        col('SK_CUSTOMER'),
        col('ID_CUSTOMER'),
        col('COD_ZIP'),
        initcap(col('DES_CITY')).alias('DES_CITY'),
        col('COD_STATE')
    )

dim_customer.write.mode('overwrite').csv('{}/dataset/e-commerce/03_dim/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [16]:
spark.stop()